In [85]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

In [22]:
data = pd.DataFrame(columns=["Time", "Setpoint", "Encoder"])

# Test 2 contains 100 step changes in setpoint and their corresponding encoder readings
with open("servoTest2.txt") as f:
    for i, line in enumerate(f):
        vs = line.split(",")
        vs = [x.split("=")[1] for x in vs]

        vs = [int(float(x)) for x in vs]

        time, setpoint, encoder = vs

        data.loc[i] = [time, setpoint, encoder]

In [96]:
data["cycleI"] = (data["Setpoint"].shift() - data["Setpoint"]).ne(0).cumsum().add(1).floordiv(2)

tg = data.groupby("cycleI")
trials = []
for name, t in tg:
    t = t.iloc[500:, :]
    step_index = t.index[t["Setpoint"].ne(t["Setpoint"].shift())]

    if len(step_index) != 2:
        print(step_index)
        break

    step_index = step_index[1]
    step_time = t["Time"][step_index]
    t["Time"] = t["Time"].sub(step_time) # set time = 0 on the step change


    trials.append(t)

Int64Index([72509], dtype='int64')


/tmp/ipykernel_184349/893453535.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
trials

[      Time  Setpoint  Encoder  cycleI
 500   -830         0        0       1
 501   -828         0        0       1
 502   -826         0        0       1
 503   -824         0        0       1
 504   -821         0        0       1
 ...    ...       ...      ...     ...
 1543  1987      1788     1786       1
 1544  1990      1788     1786       1
 1545  1993      1788     1786       1
 1546  1996      1788     1786       1
 1547  1999      1788     1786       1
 
 [1048 rows x 4 columns],
       Time  Setpoint  Encoder  cycleI
 2048  -735         0        2       2
 2049  -733         0        2       2
 2050  -731         0        2       2
 2051  -728         0        2       2
 2052  -726         0        2       2
 ...    ...       ...      ...     ...
 3051  1988      1788     1780       2
 3052  1991      1788     1780       2
 3053  1993      1788     1780       2
 3054  1996      1788     1780       2
 3055  1999      1788     1780       2
 
 [1008 rows x 4 columns],
       T

In [94]:
px.line(trials, x="Time", y=["Setpoint", "Encoder"])

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().